In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

import json
import os

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Input, Conv2DTranspose,MaxPool2D
from tensorflow.keras.layers import concatenate, Activation, MaxPooling2D, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from tensorflow.keras.utils import Sequence


from keras.metrics import MeanIoU
import tensorflow.keras.backend as K


np.random.seed(42)

In [ ]:
BASE_DIR = '/content/drive/MyDrive/Machine Learning/Clothing Co-Parsing Dataset'
im_height,im_width = (256,256)

In [ ]:
metadata_df = pd.read_csv(os.path.join(BASE_DIR,'metadata.csv'))
metadata_df = metadata_df[['image_path','label_type','label_path']]
metadata_df['image_path'] =  metadata_df['image_path'].apply(lambda im_path: os.path.join(BASE_DIR,im_path))
metadata_df['label_path'] =  metadata_df['label_path'].apply(lambda l_path: os.path.join(BASE_DIR,'labels',l_path))

#Selecting pixel level annoted dataset only
metadata_df = metadata_df[metadata_df['label_type'] == 'pixel-level']

#shuffle dataframe
metadata_df = metadata_df.sample(frac=1).reset_index(drop=True)

#train / val split 0.05
valid_df = metadata_df.sample(frac=.1,random_state=42)
train_df = metadata_df.drop(valid_df.index)

(len(valid_df),len(train_df))

(100, 904)

In [ ]:
class_dict = pd.read_csv(os.path.join(BASE_DIR,'class_dict.csv'))

classes_name = class_dict['class_name'].tolist() #series
classes_name[0] = 'none'

classes_rgb = class_dict[['r','g','b']].values.tolist() #values will be tensors
print("#classes = ",len(class_dict))
print("classes and their rgb values are " , classes_name)
print(classes_rgb)

#classes =  59
classes and their rgb values are  ['none', 'accessories', 'bag', 'belt', 'blazer', 'blouse', 'bodysuit', 'boots', 'bra', 'bracelet', 'cape', 'cardigan', 'clogs', 'coat', 'dress', 'earrings', 'flats', 'glasses', 'gloves', 'hair', 'hat', 'heels', 'hoodie', 'intimate', 'jacket', 'jeans', 'jumper', 'leggings', 'loafers', 'necklace', 'panties', 'pants', 'pumps', 'purse', 'ring', 'romper', 'sandals', 'scarf', 'shirt', 'shoes', 'shorts', 'skin', 'skirt', 'sneakers', 'socks', 'stockings', 'suit', 'sunglasses', 'sweater', 'sweatshirt', 'swimwear', 't-shirt', 'tie', 'tights', 'top', 'vest', 'wallet', 'watch', 'wedges']
[[69, 6, 90], [70, 12, 95], [71, 18, 101], [72, 25, 107], [72, 30, 112], [72, 35, 116], [71, 40, 120], [70, 47, 124], [69, 52, 127], [68, 57, 130], [66, 61, 132], [64, 67, 135], [62, 72, 136], [61, 76, 137], [58, 82, 139], [56, 86, 139], [54, 90, 140], [52, 94, 141], [50, 99, 141], [48, 103, 141], [46, 107, 142], [45, 111, 142], [43, 116, 142], [41, 120, 142], [40, 

In [ ]:
select_classes = ['null', 'accessories', 'bag', 'belt', 'blazer', 'blouse', 'bodysuit',
                  'boots', 'bra', 'bracelet', 'cape', 'cardigan', 'clogs', 'coat', 'dress',
                  'earrings', 'flats', 'glasses', 'gloves', 'hair', 'hat', 'heels', 'hoodie',
                  'intimate', 'jacket', 'jeans', 'jumper', 'leggings', 'loafers', 'necklace',
                  'panties', 'pants', 'pumps', 'purse', 'ring', 'romper', 'sandals', 'scarf',
                  'shirt', 'shoes', 'shorts', 'skin', 'skirt', 'sneakers', 'socks', 'stockings',
                  'suit', 'sunglasses', 'sweater', 'sweatshirt', 'swimwear', 't-shirt', 'tie',
                  'tights', 'top', 'vest', 'wallet', 'watch', 'wedges']

In [ ]:
len(select_classes)

59

In [ ]:
def vizualizer(images):
  '''
  #input : tensor of images
  '''
  n = len(images)
  fig , axes = plt.subplots(nrows=n,ncols=1,figsize=(8,10))
  for ax,im in zip(axes,images):
    ax.imshow(im.squeeze()/255)
  plt.show()

# One hot mask decoder
def one_hot_mask_decoder(masks,classes_rgb=classes_rgb,im_shape=(im_height,im_width)):
  # mask size = (im_h,im_w,n_classes)
  high_indexes = np.argmax(masks,axis=-1)
  color_codes =  np.array(classes_rgb)
  label = color_codes[high_indexes]
  return label


# One hot mask creator
def one_hot_mask_creator(image,classes_names=select_classes,classes_rgb=classes_rgb):
  '''
    creates indivisual one hot encoded masks for each classes

    #input : label image of type array(single mask image) - shape = (im_height,im_width,3) 
    #input : list of labels 
    #input : list of rgb values corr. to labels list
    #output : 3d mask tensor of shape (im_height,im_width,#of classes)
  '''
  (im_height,im_width,_) = image.shape
  n_classes = len(classes_names)
  # masks = np.zeros(shape=(im_height,im_width,n_classes),dtype=np.float32)
  masks = []
  for rgb in classes_rgb:
    temp = np.equal(image,rgb)
    temp = np.all(temp,axis=-1)
    masks.append(temp)
  masks = np.stack(masks,axis=-1)  
  return masks.astype(np.float32)

In [ ]:
class DataGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self, images_path_list, labels_path_list, batch_size=1, n_channels=3,im_shape=(im_height,im_width),
                 n_classes=59, shuffle=True):
        'Initialization'
        self.images_path_list = images_path_list
        self.labels_path_list = labels_path_list

        self.im_shape = im_shape
        self.batch_size = batch_size
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()
        self.indexes = np.arange(len(images_path_list))

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.images_path_list) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Get indexes
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Generate list of paths
        batch_image_paths = [self.images_path_list[k] for k in indexes]
        batch_masks_paths = [self.labels_path_list[k] for k in indexes]

        # Generate data
        X, y = self.__load_images_and_labels(batch_image_paths,batch_masks_paths)
        # assert(y.shape == (self.batch_size,self.im_shape[0],self.im_shape[1],self.n_classes))
        # assert(x.shape == (self.batch_size,self.im_shape[0],self.im_shape[1],self.n_channels))
        return X, y

    # load images
    def __load_images_and_labels(self,images_path_list,labels_path_list):
        target_size = self.im_shape
        assert(len(images_path_list)==len(labels_path_list))
        X = []
        y = []
        for img,label in zip(images_path_list,labels_path_list):
            image = img_to_array(load_img(img,target_size=target_size,color_mode="rgb",interpolation='bilinear'))
            mask = img_to_array(load_img(label,target_size=target_size,color_mode="rgb",interpolation='bilinear'))
            X.append(image) 
            y.append(one_hot_mask_creator(mask))

        return np.array(X) , np.array(y)


    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.images_path_list))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

In [ ]:
# fn , fext = os.path.splitext() # will return file name and ext of a file

def conv_2d_block(x,n_filters,k_size,batchnorm=False):
  ''' add two Conv layers with relu activation '''
  #first layer
  x = Conv2D(filters=n_filters,kernel_size=(k_size,k_size) , 
             padding='same', kernel_initializer = 'he_normal')(x)
  if batchnorm:
    x = BatchNormalization()(x)
  x = Activation('relu')(x)
  # 2nd layer
  x = Conv2D(filters=n_filters,kernel_size=(k_size,k_size) , 
              padding='same', kernel_initializer = 'he_normal')(x)
  if batchnorm:
      x = BatchNormalization()(x)
  x = Activation('relu')(x)

  return x

def get_u_net(input,n_filters=16,conv_k_size=3,pool_size=2,batchnorm=False,dropout=.2):

  c1 = conv_2d_block(input,n_filters * 1 , conv_k_size,batchnorm)
  p1 = MaxPool2D(pool_size=(pool_size,pool_size))(c1)
  p1 = Dropout(dropout)(p1)

  c2 = conv_2d_block(p1,n_filters * 2 , conv_k_size,batchnorm)
  p2 = MaxPool2D(pool_size=(pool_size,pool_size))(c2)
  p2 = Dropout(dropout)(p2)

  c3 = conv_2d_block(p2,n_filters * 4 , conv_k_size,batchnorm)
  p3 = MaxPool2D(pool_size=(pool_size,pool_size))(c3)
  p3 = Dropout(dropout)(p3)
 
  c4 = conv_2d_block(p3,n_filters * 8 , conv_k_size,batchnorm)
  p4 = MaxPool2D(pool_size=(pool_size,pool_size))(c4)
  p4 = Dropout(dropout)(p4)

  c5 = conv_2d_block(p4,n_filters * 16 , conv_k_size,batchnorm)

   #Up sampling
  u6 = Conv2DTranspose(filters=n_filters * 8 ,kernel_size=(3,3), strides=(2,2),padding='same')(c5)
  u6 = concatenate([u6,c4])
  u6 = Dropout(dropout)(u6)
  c7 = conv_2d_block(u6,n_filters * 8 , conv_k_size,batchnorm)

  u8 = Conv2DTranspose(filters=n_filters * 4 ,kernel_size=(3,3), strides=(2,2),padding='same')(c7)
  u8 = concatenate([u8,c3])
  u8 = Dropout(dropout)(u8)
  c9 = conv_2d_block(u8,n_filters * 4 , conv_k_size,batchnorm)

  u10 = Conv2DTranspose(filters=n_filters * 2,kernel_size=(3,3) , strides=(2,2),padding='same')(c9)
  u10 = concatenate([u10,c2])
  u10 = Dropout(dropout)(u10)
  c11 = conv_2d_block(u10,n_filters * 2 , conv_k_size,batchnorm)

  u12 = Conv2DTranspose(filters=n_filters * 1 ,kernel_size=(3,3), strides=(2,2),padding='same')(c11)
  u12 = concatenate([u12,c1])
  u12 = Dropout(dropout)(u12)
  c13 = conv_2d_block(u12,n_filters * 1 , conv_k_size,batchnorm)

  output = Conv2D(filters=59,kernel_size=(1,1),activation='softmax')(c13) # output layer
  model = Model(inputs=input,outputs=output,name='classifier')
  return model 

In [ ]:
import tensorflow.keras.backend as K

def DiceLoss(targets, inputs):    
    smooth=1e-6
    #flatten label and prediction tensors
    inputs = K.flatten(inputs)
    targets = K.flatten(targets)
    # BCE =  binary_crossentropy(targets, inputs)
    intersection = K.sum((targets *  inputs))    
    dice_loss = 1 - (2*intersection + smooth) / (K.sum(targets) + K.sum(inputs) + smooth)
    # Dice_BCE = dice_loss + BCE
    
    return dice_loss

In [ ]:
# Set some model compile parameters
optimizer = Adam(learning_rate=1e-4)
loss      = DiceLoss
metrics   = [MeanIoU(59),'accuracy']

# Compile our model
input = Input(shape=(im_height,im_height,3),name='img')
model = get_u_net(input)
# model.summary()

# For more GPUs
# if num_gpus > 1:
#     model = multi_gpu_model(model, gpus=num_gpus)

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:

params = {
      'batch_size' : 8,
      'n_channels' : 3,
       'n_classes' : 3,
        'im_shape' : (im_height,im_width),   
        'shuffle' : True,   
}


#generators
train_generator = DataGenerator(train_df['image_path'].values,train_df['label_path'].values,**params)
validation_generator = DataGenerator(valid_df['image_path'].values,valid_df['label_path'].values,**params)


In [ ]:

results = model.fit(x=train_generator,
          epochs=50,
           verbose=2,
          validation_data=validation_generator,
                  use_multiprocessing=True,
                    workers=6)

Epoch 1/50
113/113 - 158s - loss: 0.9914 - mean_io_u: 0.4881 - accuracy: 0.0084 - val_loss: 0.9901 - val_mean_io_u: 0.4872 - val_accuracy: 0.0097
Epoch 2/50
113/113 - 161s - loss: 0.9884 - mean_io_u: 0.4884 - accuracy: 0.0114 - val_loss: 0.9826 - val_mean_io_u: 0.4891 - val_accuracy: 0.0171
Epoch 3/50
113/113 - 201s - loss: 0.9837 - mean_io_u: 0.4892 - accuracy: 0.0161 - val_loss: 0.9821 - val_mean_io_u: 0.4892 - val_accuracy: 0.0176
Epoch 4/50
113/113 - 168s - loss: 0.9046 - mean_io_u: 0.5117 - accuracy: 0.0977 - val_loss: 0.6287 - val_mean_io_u: 0.6034 - val_accuracy: 0.3823
Epoch 5/50
113/113 - 158s - loss: 0.2924 - mean_io_u: 0.7689 - accuracy: 0.7282 - val_loss: 0.2308 - val_mean_io_u: 0.8086 - val_accuracy: 0.7916
Epoch 6/50
113/113 - 159s - loss: 0.2305 - mean_io_u: 0.8088 - accuracy: 0.7918 - val_loss: 0.2302 - val_mean_io_u: 0.8090 - val_accuracy: 0.7922
Epoch 7/50
113/113 - 165s - loss: 0.2305 - mean_io_u: 0.8088 - accuracy: 0.7918 - val_loss: 0.2303 - val_mean_io_u: 0.8089 -

In [ ]:
# how to load image efficiently??
# how to agumentation of data
# loggit??
# dice loss
# IoU score


In [ ]:
# Create Keras Pipeline for that
# results

In [ ]:
# model.summary()

In [ ]:
x = valid_df['image_path'][514]
y_true = valid_df['label_path'][514]
x = img_to_array(load_img(x,target_size=(256,256),color_mode="rgb",interpolation='bilinear'))
y_true = img_to_array(load_img(y_true,target_size=(256,256),color_mode="rgb",interpolation='bilinear'))


In [ ]:
y = model.predict(x.reshape(1,256,256,3))

In [ ]:
valid_df['image_path'].head()

In [ ]:
plt.imshow(x/255.0)

In [ ]:
plt.imshow(one_hot_mask_decoder(y).squeeze()/255)

In [ ]:
plt.imshow(y_true/255.0)
